In [3]:
from sklearn import datasets
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import  StratifiedShuffleSplit,GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator

In [4]:
mnist_path = 'mnist-original.mat'

mnist = loadmat(mnist_path)
X = mnist['data'][:5].T
y = mnist['label'][:5][0]
display(X.shape , y.shape)

# Scale all X values
scaler = StandardScaler()
X_scaled  = scaler.fit_transform(X)

sss = StratifiedShuffleSplit(n_splits=10, test_size=0.0005,train_size=0.0005, random_state=0)
train_index, test_index = next(sss.split(X=X_scaled, y=y))   

X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
display(X_train.shape,X_test.shape)

display(y_train[:5])
df_y_train = pd.get_dummies(y_train)
y_train = df_y_train.values
display(y_train[:5])

df_y_test = pd.get_dummies(y_test)
y_test = df_y_test.values


(70000, 5)

(70000,)

(35, 5)

(35, 5)

array([7., 7., 0., 1., 1.])

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [238]:
class MLPerceptron(BaseEstimator):
    
    #layers - includes input, #hidden, output
    def __init__(self,layers,eta=0,random_state=50):
        self.random_state = random_state
        self.eta = eta
        self.n_layers = len(layers)
        self.layers = layers
       
        self.init_weights()
        self.initialize_theta_weights()
        print('n_layers',self.n_layers)
        
    def initialize_theta_weights(self):
        '''
        Initialize theta_weights, initialization method depends
        on the Activation Function and the Number of Units in the current layer
        and the next layer.
        The weights for each layer as of the size [next_layer, current_layer + 1]
        '''
        print('initialize_theta_weights start')
        self.theta_weights = []
        size_next_layers = self.layers.copy()
        size_next_layers.pop(0)
        for size_layer, size_next_layer in zip(self.layers, size_next_layers):
            print(size_layer,size_next_layer)
           
            epsilon = 4.0 * np.sqrt(6) / np.sqrt(size_layer + size_next_layer)
            print('epsilon',epsilon)
            # Weigts from a uniform distribution [-epsilon, epsion]
            
            theta_tmp = epsilon * ( (np.random.rand(size_next_layer, size_layer + 1) * 2.0 ) - 1)
            print('theta_tmp',theta_tmp)
            
        self.theta_weights.append(theta_tmp)
        print(self.theta_weights)
        print('initialize_theta_weights end')
        return self.theta_weights
    
    def init_weights(self):
        #rng = np.random.RandomState(self.random_state)       
        #self.weights = np.round(rng.normal(loc=0.0,scale=0.1,size=X.shape[0] + 1),4)
        self.weights = np.array([0.5,0.2,0.3])
        
        print('X shape',X.shape[0])
        print('init_weights',self.weights.shape)
        
        return self.weights
    
    def train(self,X,y,n_iter=50):
    
        n_samples = y.shape[0]
        print('n_iter',n_iter)
        
        for it in range(n_iter):
            print('train it',it)
            self.gradients = self.back_propagation(X, y)
            """
            self.gradients_vector = self.unroll_weights(self.gradients)
            self.theta_vector = self.unroll_weights(self.theta_weights)
            self.theta_vector = self.theta_vector - self.gradients_vector
            self.theta_weights = self.roll_weights(self.theta_vector)
            """
            
    def back_propagation(self, X,y):
        
        n_samples = X.shape[0]
        print('x shape',n_samples)
    
        A, Z = self.foward_propagation(X)

        """
        # Backpropagation
        deltas = [None] * self.n_layers
        deltas[-1] = A[-1] - Y
        # For the second last layer to the second one
        for ix_layer in np.arange(self.n_layers - 1 - 1 , 0 , -1):
            theta_tmp = self.theta_weights[ix_layer]
            if self.bias_flag:
                # Removing weights for bias
                theta_tmp = np.delete(theta_tmp, np.s_[0], 1)
            deltas[ix_layer] = (np.matmul(theta_tmp.transpose(), deltas[ix_layer + 1].transpose() ) ).transpose() * self.error_part1(Z[ix_layer])

        # Compute gradients
        gradients = [None] * (self.n_layers - 1)
        for ix_layer in range(self.n_layers - 1):
            grads_tmp = np.matmul(deltas[ix_layer + 1].transpose() , A[ix_layer])
            grads_tmp = grads_tmp / n_examples
            if self.bias_flag:
                # Regularize weights, except for bias weigths
                grads_tmp[:, 1:] = grads_tmp[:, 1:] + (self.lambda_r / n_examples) * self.theta_weights[ix_layer][:,1:]
            else:
                # Regularize ALL weights
                grads_tmp = grads_tmp + (self.lambda_r / n_examples) * self.theta_weights[ix_layer]       
            gradients[ix_layer] = grads_tmp;
        return gradients       
        """
        
    def foward_propagation(self, X):
        
        print('foward_propagation n_layers',self.n_layers)
        
        inputs = [None] * self.n_layers
        nets = [None] * self.n_layers
        
        
        input_data = X
        print('foward_propagation input_data',input_data)
        
        
        for idx_layer in range(self.n_layers - 1):
            n_examples = input_data.shape[0]
            
            input_data = np.append(input_data, 1)
            print('input_data',input_data)
            
            input_data = input_data.reshape(-1,1)
            self.weights = self.weights.reshape(-1,1)
            print(self.weights.transpose().shape,input_data.shape)
           
            
            inputs[idx_layer] = input_data
            
            nets[idx_layer + 1] = self.net(input_data)
           
            output_data = self.sigmoid(nets[idx_layer + 1])
            
            print('[foward_propagation] layer',idx_layer + 1, ' net ', nets[idx_layer + 1], 'output',output_data)
                
            input_data = output_data

        #A[self.n_layers - 1] = output_layer
               
        return inputs, nets
        

    
    def predict(self,X,y):
        pass
    
    def net(self,X):
        return np.dot(self.weights.transpose(),X)
    
    def sigmoid(self,value):
        sig = 1 / (1 + np.exp(-value))
        return sig     
 
    def error_part1(self, value):
        err_part1 = self.sigmoid(value) * (1 - seld.sigmoid(value)) 
        return err_part1
   
 
       

In [239]:
X = pd.DataFrame([1,0])
print(X.values.shape)
y = pd.DataFrame([1,0])
print(y.values.shape)


(2, 1)
(2, 1)


In [240]:
mlp = MLPerceptron(layers = [2, 3,2, 1],eta=0,random_state=50) #[784, 784, 10]

mlp.train(X.values, y.values,n_iter=1)
    
    

X shape 2
init_weights (3,)
initialize_theta_weights start
2 3
epsilon 4.381780460041329
theta_tmp [[ 0.09874648  4.28090146  3.01591852]
 [ 0.07189939 -2.55694854  3.52028905]
 [-2.20576646  1.83710336 -2.2290185 ]]
3 2
epsilon 4.381780460041329
theta_tmp [[ 3.45218532 -3.6676145   1.16879888 -2.92929289]
 [ 2.76087586 -3.75604021 -1.31833965 -1.63146981]]
2 1
epsilon 5.65685424949238
theta_tmp [[-2.92826741 -3.53841589 -1.86142306]]
[array([[-2.92826741, -3.53841589, -1.86142306]])]
initialize_theta_weights end
n_layers 4
n_iter 1
train it 0
x shape 2
foward_propagation n_layers 4
foward_propagation input_data [[1]
 [0]]
input_data [1 0 1]
(1, 3) (3, 1)
[foward_propagation] layer 1  net  [[0.8]] output [[0.68997448]]
input_data [0.68997448 1.        ]
(1, 3) (2, 1)


ValueError: shapes (1,3) and (2,1) not aligned: 3 (dim 1) != 2 (dim 0)

In [1]:
# -*- coding: utf-8 -*-
"""
Parameters of MLP  #  MULTI LAYER PERCEPTRON
Number of layers : 3 (input, hidden1, output)  כמה שכבות קלט נסתרת ופלט
Elements in layers : [784, 100, 10] # 28*28 = 784 לוקחים את התמונה ופורסים אותה בווקטור אחד     # 100 נוריונים # השכבה האחרונה היא סופטמקס המתצמתת את המאה נוירונים ומקבלים עשרה מספרים את המאה אפשר לשחק עם זה
Activation function : Rectified Linear function
Regularization parameter : 0 
Bias element added in input layers : False  # אפשר להוסיף ביס ואפשר בלי

This file contains the Multi-Layer Perceptron (MLP) class which creates a
fully-connected-feedforward-artifitial-neural-network object with methods
for its usage
Methods:
    __init__()
    train(X, y, iterations, reset)
    predict(X)
    initialize_theta_weights()
    backpropagation(X, Y)
    feedforward(X)
    unroll_weights(rolled_data)
    roll_weights(unrolled_data)
    sigmoid(z)
    relu(z)
    sigmoid_derivative(z)
    relu_derivative(z)
"""
import numpy as np

class Mlp(): # זה קלאס MLP
   
     

    def predict(self, X):
        '''
        Given X (feature matrix), y_hay is computed
        Arguments:
            X      : Feature matrix [n_examples, n_features]
        Output:
            y_hat  : Computed Vector Class for X    # עבור כל איטרציה נקבל וואי כובע
        '''
        A , Z = self.feedforward(X)
        Y_hat = A[-1]
        return Y_hat

      

  

    def unroll_weights(self, rolled_data):
        '''
        Unroll a list of matrices to a single vector
        Each matrix represents the Weights (or Gradients) from one layer to the next
        '''
        unrolled_array = np.array([])
        for one_layer in rolled_data:
            unrolled_array = np.concatenate((unrolled_array, one_layer.flatten(1)) )
        return unrolled_array

    def roll_weights(self, unrolled_data):
        '''
        Unrolls a single vector to a list of matrices
        Each matrix represents the Weights (or Gradients) from one layer to the next
        '''
        size_next_layers = self.size_layers.copy()
        size_next_layers.pop(0)
        rolled_list = []
        if self.bias_flag:
            extra_item = 1
        else:
            extra_item = 0
        for size_layer, size_next_layer in zip(self.size_layers, size_next_layers):
            n_weights = (size_next_layer * (size_layer + extra_item))
            data_tmp = unrolled_data[0 : n_weights]
            data_tmp = data_tmp.reshape(size_next_layer, (size_layer + extra_item), order = 'F')
            rolled_list.append(data_tmp)
            unrolled_data = np.delete(unrolled_data, np.s_[0:n_weights])
        return rolled_list

   
 

def plot_digits(X, y, num_digits_to_plot=3):
    for i in range(num_digits_to_plot):
        _image = X[i,:]
        _label = y[i]

        # 784 columns correspond to 28x28 image
        plottable_image = np.reshape(_image, (28, 28))

        # Plot the image
        plt.figure()
        plt.imshow(plottable_image, cmap='gray_r')
        plt.title('Digit Label: {}'.format(_label))
        plt.show()

# Load data
X, Y, labels, y = load_mnist()
plot_digits(X, y, num_digits_to_plot=5)
print('num examples by num of features', X.shape)

tic = time.time()
epochs = 17 
"""enter the number of epochs for training "EP" """ 
loss = np.zeros([epochs,1])
# Creating the MLP object initialize the weights
mlp_classifier = Mlp(size_layers = [784, 777, 10], 
                     #"""enter the number of neurons in hidden layer "NN" """, 10], 
                         act_f   = 'relu',
                         reg_lambda  = 0,
                         bias_flag   = False)

for ix in range(epochs):
    mlp_classifier.train(X, Y, 17)
                       #  """Number of times the back-propagation algorithm is performed for each epoch "BP"  """)
    Y_hat = mlp_classifier.predict(X)
    # loss
    loss[ix] = (0.5)*np.square(Y_hat - Y).mean()

print(str(time.time() - tic) + ' s')
        
# Ploting loss vs epochs
plt.figure()
ix = np.arange(epochs)
plt.plot(ix, loss)

# Training Accuracy
Y_hat = mlp_classifier.predict(X)
y_tmp = np.argmax(Y_hat, axis=1)
y_hat = labels[y_tmp]

acc = np.mean(1 * (y_hat == y))
print('Training Accuracy: ' + str(acc*100) + '%')


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 45)

In [ ]:
def mlp(data,target,n_hidden_layers=1,n_hidden_len = 2,n_iter=50):
    network = {} 
    outputs = {}
        
    all_layers = n_hidden_layers + 1 #includes output layer
    
    outputs = np.zeros(n_hidden_len)
    
    print('all_layers , includes output layer', all_layers)
        
    data = data[:n_iter]
    for idx, sample in enumerate(data):
        for layer in range(1,all_layers+1,1):
            for hidden_len in range(n_hidden_len):
                #print('sample',sample,'idx',idx,'layer',layer+1,'layer_len',layer_len)
                print('layer',layer,'Perceptron',hidden_len)
                if layer == 1:
                    P = Perceptron(42).fit(sample,target[idx])
                    print('[O_]',P.O_)
                    #key = str(layer) + ' ' + str(hidden_len)
                    network[layer] = P
                    outputs.append(P.O_)
                    
                else:
                    pass
                    #print('layer',layer)
                    #print(outputs[layer])
                    
                    #network[layer+1] = Perceptron(42).fit(np.array(outputs),target[idx])
                #propagate the inputs forward
                #propagate_forward(layer,sample)